# Carga y Validación de Datos

In [ ]:
# Bloque autocontenible
import subprocess, sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

for pkg in ['pandas', 'numpy', 'openpyxl', 'pyarrow']:
    try: __import__(pkg)
    except: subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print(f' PROJECT_ROOT: {PROJECT_ROOT}')

✓ PROJECT_ROOT: c:\Proyecto_Enfermedades_Alto_Costo completo\Proyecto_Enfermedades_Alto_Costo completo


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

## 1. Carga del Dataset

In [ ]:
file_path = DATA_RAW / 'Mortalidad_2019_colombia1.xlsx'
df = pd.read_excel(file_path, sheet_name='BD FINAL')
print(f' Dataset: {df.shape[0]} × {df.shape[1]}')
df.head()

✓ Dataset: 42644 × 37


,COD_DPTO,Nom Dep,COD_MUNIC,A_DEFUN,SIT_DEFUN,TIPO_DEFUN,ANO,MES,HORA,MINUTOS,...,CONS_EXP,ASIS_MED,CAUSA_MULT,C_BAS1,NOMBRE CIE-10,CUENTA DE ALTO COSTO,GRUPO,CAUSA_667,IDPROFCER,CAU_HOMOL
0,17,CALDAS,1,1,1,2,2019,12,21,10,...,1,1,C712,C712,Tumor maligno del lobulo temporal,Tumor maligno del lobulo temporal,CANCER,214,1,31
1,19,CAUCA,1,1,3,2,2019,12,5,0,...,2,2,C719,C719,"Tumor maligno del encefalo, parte no especificada","Tumor maligno del encefalo, parte no especificada",CANCER,214,1,31
2,73,TOLIMA,168,1,1,2,2019,12,23,50,...,1,1,I219/I139/I509/N189,N189,"Enfermedad renal cronica, no especificada","INSUFICIENCIA RENAL CRONICA, NO ESPECIFICADA",ER CRONICA,610,1,74
3,23,CÓRDOBA,1,1,1,2,2019,12,12,33,...,1,1,J80/C780 C800*J449,C800,"Tumor maligno, de sitio primario desconocido, ...","Tumor maligno de sitio primario desconocido, a...",CANCER,214,1,34
4,8,ATLÁNTICO,1,1,1,2,2019,11,15,15,...,2,1,I461/R570/I313/J90*N185,N185,"Enfermedad renal crónica, etapa 5",ENFERMEDAD RENAL CRÓNICA EN ESTADIO 5,ER CRONICA,610,1,74


## 2. Diccionario

In [ ]:
df_dict_raw = pd.read_excel(file_path, sheet_name='DICCIONARIO')
dict_data = []
for idx, row in df_dict_raw.iterrows():
    content = str(row.iloc[0])
    if content.startswith('V') and len(content.split()) >= 2:
        parts = content.split(maxsplit=2)
        if len(parts) >= 2:
            dict_data.append({'ID': parts[0], 'VARIABLE': parts[1], 'DESCRIPCION': parts[2] if len(parts) > 2 else ''})
df_dict = pd.DataFrame(dict_data)
df_dict.to_parquet(DATA_PROCESSED / 'dictionary.parquet', index=False)
print(f' Diccionario: {len(df_dict)} variables')
df_dict.head(10)

✓ Diccionario: 56 variables


,ID,VARIABLE,DESCRIPCION
0,V1338,COD_DPTO,Departamento donde discrete numeric ...
1,V1339,COD_MUNIC,Municipio donde discrete numeri...
2,V1340,A_DEFUN,Área donde ocurrió la discrete numeric ...
3,V1341,SIT_DEFUN,Sitio donde ocurrió la discrete numeric...
4,V1342,OTRSITIODE,"Otro sitio, ¿cúal? discrete nume..."
5,V1343,TIPO_DEFUN,Tipo de defunción discrete numeri...
6,V1344,ANO,Año en que ocurrió la discrete numeric ...
7,V1345,MES,Mes en que ocurrió la discrete numeric ...
8,V1346,HORA,Hora en que ocurrió la discrete numeric ...
9,V1347,MINUTOS,Minutos en que ocurrió discrete numeric ...


## 3. Calidad de Datos

In [5]:
dups = df.duplicated().sum()
print(f'Duplicados: {dups}')
if dups > 0: df = df.drop_duplicates()

null_info = pd.DataFrame({'Columna': df.columns, 'Nulos': df.isnull().sum().values, 'Porcentaje': (df.isnull().sum() / len(df) * 100).values})
null_info = null_info[null_info['Nulos'] > 0].sort_values('Porcentaje', ascending=False)
print(null_info)

Duplicados: 0
        Columna  Nulos  Porcentaje
17   SIMUERTEPO  42609   99.917925
25  IDADMISALUD   1026    2.405966
18    OCUPACION    311    0.729294
16   MUERTEPORO    165    0.386924
22     CODMUNRE    151    0.354094
21     CODPTORE    151    0.354094
23     AREA_RES    150    0.351749
20      CODPRES      9    0.021105


In [ ]:
null_pct = df.isnull().sum() / len(df)
cols_high_null = null_pct[null_pct > 0.50].index.tolist()
if cols_high_null:
    print(f'Eliminando {len(cols_high_null)} columnas con >50% nulos')
    df = df.drop(columns=cols_high_null)
print(f' Columnas: {df.shape[1]}')

Eliminando 1 columnas con >50% nulos
✓ Columnas: 36


## 4. Variable Objetivo

In [7]:
target_col = 'GRUPO'
target_dist = df[target_col].value_counts()
target_pct = (df[target_col].value_counts(normalize=True) * 100).round(2)
print(pd.DataFrame({'Clase': target_dist.index, 'Frecuencia': target_dist.values, 'Porcentaje': target_pct.values}))
print(f'\nDesbalanceo: {target_dist.iloc[0] / target_dist.iloc[-1]:.1f}:1')

        Clase  Frecuencia  Porcentaje
0      CANCER       38028       89.18
1  ER CRONICA        2584        6.06
2         VIH        1937        4.54
3  HEMOFILIA           95        0.22

Desbalanceo: 400.3:1


## 5. Data Leakage

In [ ]:
patterns = ['CUENTA', 'NOMBRE', 'CIE', 'CAUSA', 'C_BAS', 'CAU_', 'CONS_', 'IDPROFCER', 'ASIS_MED']
suspicious = [col for col in df.columns if col != target_col and any(p in col.upper() for p in patterns)]
if suspicious:
    print(f'Eliminando {len(suspicious)} columnas sospechosas')
    df = df.drop(columns=suspicious)
print(f' Final: {df.shape[0]} × {df.shape[1]}')

Eliminando 9 columnas sospechosas
✓ Final: 42644 × 27


In [ ]:
df.to_parquet(DATA_PROCESSED / 'raw_clean.parquet', index=False)
print(f' Guardado: {df.shape}')

✅ Guardado: (42644, 27)


Análisis: se Cargo el dataset original y se confirmo la dimensión inicial 42644 × 37. se elimino 1 columna con >50% de nulos y 9 columnas catalogadas como sospechosas; la tabla final quedó en 42644 × 27. La variable objetivo usada es GRUPO.


Conclusiones: La limpieza estructural dejó una base consistente y trazable (42644 registros, 27 columnas) lista para análisis posteriores y modelado.